In [1]:
import numpy as np
import pandas as pd
import requests
import time
import os
import re
import html
import ast

import jsonlines

import wand.image

import cv2 as cv
import json

### Read data

In [ ]:
data = pd.read_csv('concatenatedProlife_new.csv').astype(str)

### Function to make GET call

In [ ]:
def getUserData(ids):
    headers = {'Authorization':'Bearer '}
    url = "https://api.twitter.com/2/users"
    params = {
        'ids':ids,
        'user.fields':'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,url,username,verified,withheld',
        'tweet.fields':'attachments,author_id,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,lang'
    }
    
    r = requests.get(url=url, params=params, headers=headers)
    
    return r.json()

In [ ]:
data.shape[0]

### Make calls on a hopping window of 100

In [ ]:
start = 0
collection = []
callCount = 0

while start < data.shape[0]:
    end = start + 100
    ids = ','.join(data.iloc[start:end]['author_id'])
    collection.extend(getUserData(ids)['data'])
    start = end
    
    callCount += 1
    if callCount % 49 == 0:
        print(f"Sleeping for few minutes, {callCount} calls made.")
        time.sleep(1000)

user_data = pd.DataFrame(collection)

In [ ]:
temp = pd.read_csv('ConcatenatedUsers.csv')
temp2 = pd.concat([temp,user_data])

In [ ]:
temp2.drop_duplicates(['id']).to_csv('ConcatenatedUsers.csv',index=False)

In [ ]:
#user_data.to_csv('ConcatenatedUsers.csv',index=False)

### Get profile images

In [13]:
user_data = pd.read_csv('ConcatenatedUsers.csv')

In [ ]:
calls = 19100

for index, user in user_data.iloc[19100:].iterrows():
    url = str(user['profile_image_url'])
    userid = str(user['id'])
    
    try:
        img_data = requests.get(url).content
    except:
        continue
    
    with open(f'profile_images/user_{userid}.jpg', 'wb') as handler:
        handler.write(img_data)
    
    calls += 1
    if (calls % 100 == 0):
        print(f'Calls made = {calls}')

### Create JSON Lines file

In [3]:
user_data.head()

,description,created_at,profile_image_url,name,protected,verified,url,username,id,location,entities,pinned_tweet_id
0,"(he/they) For those who need, we must do what ...",2014-09-20T18:53:50.000Z,https://pbs.twimg.com/profile_images/145316546...,MintyFreshness,False,False,NaN,Intensefreshnes,2822594617,The cum zone,NaN,NaN
1,"📍 Dallas, TX I show you where the adventure is...",2010-06-09T02:39:11.000Z,https://pbs.twimg.com/profile_images/143451186...,Dallas Latina Foodie,False,False,https://t.co/IfrZpDpvPs,allycelebrates,153642560,"Dallas, TX","{'url': {'urls': [{'start': 0, 'end': 23, 'url...",1.450854e+18
2,#FBR #FuckTrump #TaxTheRich #BlackLivesMatter ...,2015-09-04T16:18:33.000Z,https://pbs.twimg.com/profile_images/137127092...,DMoney,False,False,NaN,dmoney31267,3449791700,"Colorado, USA","{'description': {'hashtags': [{'start': 0, 'en...",NaN
3,Stock Investor / Anytime Fitness Member,2015-10-21T23:30:07.000Z,https://pbs.twimg.com/profile_images/749396738...,Ray Blackwell,False,False,NaN,RayBlackwell7,4012849515,"Warren, PA",NaN,NaN
4,press secretary for all gay people. digital gr...,2019-05-31T19:15:03.000Z,https://pbs.twimg.com/profile_images/144449859...,kaitlyn 🇬🇹,False,False,https://t.co/Y8aIhEM8jN,kaitlynsolares,1134538801680977920,Boston 🐀,"{'url': {'urls': [{'start': 0, 'end': 23, 'url...",NaN


In [78]:
json_dicts = []

for _, row in user_data.iterrows():
    
    # Preprocess description
    description = str(row['description'])
    description = re.sub(r"(?:\@|https?\://)\S+", "", description)
    description = html.unescape(description)
    description = re.sub('[^A-Za-z ]+', '', description)
    description = description.strip().lower()
    description = ' '.join(description.split())
    
    # Preprocess name
    name = str(row['name'])
    name = re.sub(r"(?:\@|https?\://)\S+", "", name)
    name = html.unescape(name)
    name = re.sub('[^A-Za-z ]+', '', name)
    name = name.strip().lower()
    name = ' '.join(name.split())
    
    # Preprocess username
    username = str(row['username'])
    username = re.sub(r"(?:\@|https?\://)\S+", "", username)
    username = html.unescape(username)
    username = re.sub('[^A-Za-z ]+', '', username)
    username = username.strip().lower()
    username = ' '.join(username.split())
    
    
    if not os.path.isfile(f'profile_images/user_{row["id"]}.jpg'):
        img = '/content/profile_images/missing_image.jpg'
    else:
        img = f'/content/profile_images/user_{row["id"]}.jpg'
    
    user_dict = {
        "id": str(row['id']),
        "name": name,
        "screen_name": username,
        "description": description,
        "lang": "en",
        "img_path":img
    }
    
    json_dicts.append(user_dict)

In [79]:
# with open("data.jsonl", "w", encoding="utf-8") as text_file:
#     text_file.write(jsonstring)

with jsonlines.open('data.jsonl', 'w') as writer:
    writer.write_all(json_dicts)

### Find grayscale images

In [67]:
collect_graysc = []

for _, row in user_data.iterrows():
    url = f'profile_images_orig/user_{row["id"]}.jpg'
    new_url = f'profile_images/user_{row["id"]}.jpg'
    if os.path.isfile(new_url):
        img = cv.imread(new_url)
        w,h,_ = img.shape
        flag = True
        for i in range(w):
            
            if not flag:
                break
            
            for j in range(h):
                r, g, b = img[i][j]
                if r == g == b: 
                    pass
                else:
                    flag = False
                    break
        
        if flag:
            print(row['id'])
            collect_graysc.append(row["id"])

884796902
1422367882592333828
213550146
1321909202147790848
827523821385158657
465829056
946226966075604992
569597530
766719206247108608
2361636319
21486053
4226402099
126429617
904570926766968832
125508260
1108802500600217600
1098364699618488320
82689705
271432134
840623059493036032
953028109363707904
16034381
1342605352550936576
1334935874052636674
13323862
1420838064356827147
1293934080518750208
1247549077
398521222
763382314629992448
302917923
23790303
39005137
18809159
1352234446402048003
606468633
961931322536099841
1220063479287037952
149476790
467709287
1286414399738122245
16029337
17091121
1205549741876101128
24702075
1333283589291601923
1368716742197846017
18275679
1184333019101794304
1108559620128178176
341889397
1089585736914010113
1022583292959309830
934892606
2748315394
1967788057
623162659
20217818
253386630
36161030
247379602
25324758
1381439179398115331
222467843
2800609133
821370971940651009
1228916520539082752
1557132781
1340379798720315393
1431134517046259712
100157

30564747
397435844
14296548
4862036988
2724662658
39980887
1169739725525135360
1038290324706209793
978454670111887360
34180814
2879261147
2768854922
298865656
15993333
1274850030998876160
166830366
47723252
754770694190297088
3257207293
3314301108
917143626785763328
1390407282559619075
3740353213
753786120488493056
2979949737
2885929432
3979287796
333472041
3223318914
795120763938320391
1307572097745596416
778556883107995649
97739502
946501808557617152
1363918699430776833
50666280
840008209448480770
3486910214
1432091337151684617
1381717606030266377
1280197044309823488
1028620936176656384
1333636517403627520
2766016889
1014249685
1424920576443965442
1368864790655950848
172475589
309306903
1042617649732222976
1425463975681003525
785730007
94206208
237888723
1250107355800322048
3527774236
3934221
109048539
1367653471
1274391600345513985
303593747
390248849
1241517068329127938
2681282750
1346533548099039233
273965642
838636040
15730608
11859472
821262683123175425
2287543800
497157573
1174

14899110
326552479
36996153
1164723425530830849
923944886541283329
938209186457849856
19996935
766057748316422147
924233875592802304
1092448827255083008
18001922
217545214
757736003167588352
1142714188290121728
1302185071
1329501169484509193
1341674815


In [68]:
len(collect_graysc)

1167

### Remove alpha channel

In [4]:
def hasAlpha(image_path):
    with wand.image.Image(filename=image_path) as img:
        alpha = img.alpha_channel
        return alpha

In [76]:
def removeAlpha(image_path, new_image_path):
    with wand.image.Image(filename=image_path) as img:
        img.alpha_channel = 'remove' 
        img.background_color = wand.image.Color('white')
        img.save(filename=new_image_path)

In [77]:
for _, row in user_data.iterrows():
    url = f'profile_images_orig/user_{row["id"]}.jpg'
    new_url = f'profile_images/user_{row["id"]}.jpg'
    if os.path.isfile(url):
        try:
            if row['id'] not in collect_graysc:
                removeAlpha(url, new_url)
        except:
            pass

### Find image size mismatch

In [80]:
for _, row in user_data.iterrows():
    new_url = f'profile_images/user_{row["id"]}.jpg'
    if os.path.isfile(new_url):
        img = cv.imread(new_url)
        w,h,_ = img.shape
        if w != 48 or h != 48:
            print(row['id'])

1344940193317470208
1191891248044544000
300885139
14167388
597779527


### Read M3 outputs

In [3]:
f = open('m3_data.json')
m3_data = json.load(f)

In [8]:
m3_data = pd.DataFrame(m3_data).transpose().reset_index().rename(columns={'index':'id'})
m3_data.head()

,id,gender,age,org
0,2822594617,"{'male': 0.7122, 'female': 0.2878}","{'<=18': 0.261, '19-29': 0.2823, '30-39': 0.01...","{'non-org': 0.9664, 'is-org': 0.0336}"
1,153642560,"{'male': 0.0396, 'female': 0.9604}","{'<=18': 0.2707, '19-29': 0.5784, '30-39': 0.1...","{'non-org': 0.9998, 'is-org': 0.0002}"
2,3449791700,"{'male': 0.9974, 'female': 0.0026}","{'<=18': 0.1214, '19-29': 0.2452, '30-39': 0.0...","{'non-org': 0.9997, 'is-org': 0.0003}"
3,4012849515,"{'male': 0.9995, 'female': 0.0005}","{'<=18': 0.011, '19-29': 0.1272, '30-39': 0.00...","{'non-org': 0.988, 'is-org': 0.012}"
4,1134538801680977920,"{'male': 0.0005, 'female': 0.9995}","{'<=18': 0.2342, '19-29': 0.7656, '30-39': 0.0...","{'non-org': 1.0, 'is-org': 0.0}"


In [11]:
def getMostProbableKey(x):
    maxVal = 0
    maxKey = None
    for key in x.keys():
        if x[key] > maxVal:
            maxVal = x[key]
            maxKey = key
    return maxKey

m3_data['gender'] = m3_data['gender'].apply(getMostProbableKey)
m3_data['age'] = m3_data['age'].apply(getMostProbableKey)
m3_data['org'] = m3_data['org'].apply(getMostProbableKey)
m3_dat

In [19]:
user_data['id'] = user_data['id'].astype(str)

In [21]:
concatenatedUsers_m3 = pd.merge(user_data,m3_data,on='id')
concatenatedUsers_m3.to_csv('ConcatenatedUsers_m3.csv',index=False)

KeyError: "None of [Index(['state', 'city', 'population_density'], dtype='object')] are in the [columns]"

In [4]:
all_data = pd.read_csv('all_data.csv')
all_data['id']=all_data['id'].astype(str)
f = open('sentiment.txt','r')
sent_collection = ast.literal_eval(f.readlines()[0])

In [5]:
all_data['sentiment']=all_data.id.map(sent_collection)

In [8]:
all_data.to_csv('all_data.csv',index=False)